# Libraries

In [1]:
import sys
import numpy as np
import scipy.stats
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3
import json
import xml.etree.ElementTree as et
import os

np.random.seed(123)

# plot style
plt.style.use("seaborn-v0_8")
# print(plt.style.available)

# the "R4" palette
_colours = ["#000000f0", "#DF536Bf0", "#61D04Ff0", "#2297E6f0",
            "#28E2E5f0", "#CD0BBCf0", "#F5C710f0", "#999999f0"]

_linestyles = ["solid", "dashed", "dashdot", "dotted"]

# A different plotting style for each plotted line
plt.rcParams["axes.prop_cycle"] = plt.cycler(color=_colours, linestyle=_linestyles*2)
plt.rcParams["patch.facecolor"] = _colours[0]

# Python version
print(f"sys: {sys.version}")
print(f"numpy: {np.__version__}")
print(f"scipy: {scipy.__version__}")
print(f"pandas: {pd.__version__}")
print(f"matplotlib: {matplotlib.__version__}")
print(f"seaborns {sns.__version__}")

sys: 3.9.5 (tags/v3.9.5:0a7dcbd, May  3 2021, 17:27:52) [MSC v.1928 64 bit (AMD64)]
numpy: 1.24.2
scipy: 1.10.1
pandas: 1.5.3
matplotlib: 3.7.1
seaborns 0.12.2


# Load data

In [2]:
def read_files(file_name):
    dbfile = os.path.join(os.getcwd(), "data", "travel_stackexchange_com_2017", file_name)
    return pd.read_csv(dbfile, comment="#")

In [3]:
Tags = read_files("Tags.csv.gz")
Tags.head(5)

,Count,ExcerptPostId,Id,TagName,WikiPostId
0,104,2138.0,1,cruising,2137.0
1,43,357.0,2,caribbean,356.0
2,43,319.0,4,vacations,318.0
3,8,14548.0,6,amazon-river,14547.0
4,100,1792.0,8,romania,1791.0


In [4]:
Users = read_files("Users.csv.gz")
Users.head(5)

,AccountId,Age,CreationDate,DisplayName,DownVotes,Id,LastAccessDate,Location,Reputation,UpVotes,Views
0,-1.0,NaN,2011-06-21T15:16:44.253,Community,12646,-1,2011-06-21T15:16:44.253,on the server farm,1.0,2472.0,0.0
1,2.0,40.0,2011-06-21T20:10:03.720,Geoff Dalgas,0,2,2016-05-29T01:18:20.767,"Corvallis, OR",101.0,1.0,31.0
2,7598.0,32.0,2011-06-21T20:11:02.490,Nick Craver,0,3,2015-09-14T12:03:40.247,"Winston-Salem, NC",101.0,1.0,14.0
3,1998.0,31.0,2011-06-21T20:12:21.223,Emmett,0,4,2013-03-01T22:30:26.790,"San Francisco, CA",101.0,1.0,10.0
4,29738.0,NaN,2011-06-21T20:15:31.410,Kevin Montrose,0,5,2017-02-07T02:24:34.733,"New York, NY, United States",101.0,1.0,11.0


In [5]:
Badges = read_files("Badges.csv.gz")
Badges.head(5)

,Class,Date,Id,Name,TagBased,UserId
0,3,2011-06-21T20:16:48.910,1,Autobiographer,False,2
1,3,2011-06-21T20:16:48.910,2,Autobiographer,False,3
2,3,2011-06-21T20:16:48.910,3,Autobiographer,False,4
3,3,2011-06-21T20:21:49.157,4,Autobiographer,False,6
4,3,2011-06-21T20:21:49.157,5,Autobiographer,False,8


In [6]:
Posts = read_files("Posts.csv.gz")
Posts.head(5)

,AcceptedAnswerId,AnswerCount,ClosedDate,CommentCount,CommunityOwnedDate,CreationDate,FavoriteCount,Id,LastActivityDate,LastEditDate,LastEditorUserId,OwnerUserId,ParentId,PostTypeId,Score,Title,ViewCount
0,393.0,4.0,2013-02-25T23:52:47.953,4,NaN,2011-06-21T20:19:34.730,NaN,1,2012-05-24T14:52:14.760,2011-12-28T21:36:43.910,101.0,9.0,NaN,1,8,What are some Caribbean cruises for October?,419.0
1,NaN,7.0,NaN,4,NaN,2011-06-21T20:22:33.760,5.0,2,2014-05-05T22:30:31.273,2012-09-03T16:56:27.963,13.0,13.0,NaN,1,32,How can I find a guide that will take me safel...,1399.0
2,NaN,NaN,NaN,2,NaN,2011-06-21T20:24:28.080,NaN,3,2011-06-21T20:24:28.080,NaN,NaN,9.0,2.0,2,12,NaN,NaN
3,NaN,1.0,NaN,1,NaN,2011-06-21T20:24:57.160,NaN,4,2013-01-09T09:55:22.743,2013-01-09T09:55:22.743,693.0,24.0,NaN,1,8,Does Singapore Airlines offer any reward seats...,243.0
4,770.0,5.0,NaN,0,NaN,2011-06-21T20:25:56.787,2.0,5,2012-10-12T20:49:08.110,2011-12-28T21:36:18.230,101.0,13.0,NaN,1,14,What is the easiest transportation to use thro...,395.0


In [7]:
Votes = read_files("Votes.csv.gz")
Votes.head(5)

,BountyAmount,CreationDate,Id,PostId,UserId,VoteTypeId
0,NaN,2011-06-21T00:00:00.000,1,1,NaN,2
1,NaN,2011-06-21T00:00:00.000,2,1,NaN,2
2,NaN,2011-06-21T00:00:00.000,3,2,NaN,2
3,NaN,2011-06-21T00:00:00.000,4,3,NaN,2
4,NaN,2011-06-21T00:00:00.000,5,5,13.0,5


# Exporting Data to a Database

In [8]:
# Create path to the file
dbfile = os.path.join(os.getcwd(), "data", "sqlite_db", "travel.db")
print(dbfile)

c:\Users\danie\Desktop\DSR - book\code\data\sqlite_db\travel.db


In [9]:
# Connect to the database
conn = sqlite3.connect(dbfile)

In [10]:
# Export pandas `DataFrame` them to the database
# NOTE: this must be done one once!
write_file = False
if write_file:
    # use the followin lines only to write the file the first time
    Tags.to_sql("Tags", conn, index=False)
    Users.to_sql("Users", conn, index=False)
    Badges.to_sql("Badges", conn, index=False)
    Posts.to_sql("Posts", conn, index=False)
    Votes.to_sql("Votes", conn, index=False)

# Pandas & SQL

In [11]:
# Select all columns from "Tags", first 3 rows
pd.read_sql_query(
    """
    SELECT * FROM Tags LIMIT 3
    """,
    conn) 

# type: ignore

,Count,ExcerptPostId,Id,TagName,WikiPostId
0,104,2138.0,1,cruising,2137.0
1,43,357.0,2,caribbean,356.0
2,43,319.0,4,vacations,318.0


In [12]:
# From "Tags", select two columns "TagName" and "Count",
# and rows for which "TagName" is equal to 'poland', 'australia', or 'china'
pd.read_sql_query(
    """
    SELECT TagName, Count
    FROM Tags
    WHERE TagName IN ('poland', 'australia', 'china')
    """,
    conn
)


,TagName,Count
0,china,443
1,australia,411
2,poland,139


In [13]:
# Select columns "Title", "Score", "ViewCount", FavoriteCount
# from "Posts" whose rows fulfil these conditions:
# "PostTypeId" is equal to 1
# "ViewCount" equal to or larger than 10000
# "FavoriteCount" is between 35 and 100
pd.read_sql_query(
    """
    SELECT Title, Score, ViewCount, FavoriteCount
    FROM Posts
    WHERE PostTypeId=1 AND
        ViewCount>=10000 AND
        FavoriteCount BETWEEN 35 AND 100
    """,
    conn
)

,Title,Score,ViewCount,FavoriteCount
0,When traveling to a country with a different c...,136,16838.0,35.0
1,"How can I do a ""broad"" search for flights?",95,33554.0,49.0
2,Tactics to avoid getting harassed by corrupt p...,156,13220.0,42.0
3,Flight tickets: buy two weeks before even duri...,109,49440.0,36.0
4,"OK we're all adults here, so really, how on ea...",306,73808.0,79.0
5,How to intentionally get denied entry to the U...,219,63873.0,53.0
6,How do you know if Americans genuinely/literal...,254,30312.0,79.0
7,"OK, we are all adults here, so what is a bidet...",140,52265.0,38.0
8,How to cope with too slow Wi-Fi at hotel?,82,16380.0,41.0


In [14]:
# Select columns "Title" and "Score"
# from "Posts" whose rows fulfil these conditions:
# "ParentId" is missing
# "Title" is well-defined.
# Then, sort the results by the "Score" column (descending order).
# Finally, return only the first five rows.
pd.read_sql_query(
    """
    SELECT Title, Score
    From Posts
    WHERE ParentId is NULL AND Title IS NOT NULL
    ORDER BY Score DESC
    LIMIT 5
    """,
    conn
)

,Title,Score
0,"OK we're all adults here, so really, how on ea...",306
1,How do you know if Americans genuinely/literal...,254
2,How to intentionally get denied entry to the U...,219
3,Why are airline passengers asked to lift up wi...,210
4,Why prohibit engine braking?,178


In [15]:
# From "Badges", select all unique "Name" values for "UserId" equal to 23
pd.read_sql_query(
    """
    SELECT DISTINCT Name
    FROM Badges
    WHERE UserId=23
    """,
    conn
)

,Name
0,Supporter
1,Student
2,Teacher
3,Scholar
4,Beta
5,Nice Question
6,Editor
7,Nice Answer
8,Yearling
9,Popular Question


In [16]:
# From "Badges", select all unique pairs "Name" and "Date",
# for "UserId" equal to 23,
# Then, extract the year and store it in a new column named "Year".
pd.read_sql_query(
    """
    SELECT DISTINCT 
        Name,
        CAST(strftime('%Y', Date) AS FLOAT) AS Year
    FROM Badges
    WHERE UserId=23
    
    """,
    conn
)

,Name,Year
0,Supporter,2011.0
1,Student,2011.0
2,Teacher,2011.0
3,Scholar,2011.0
4,Beta,2011.0
5,Nice Question,2011.0
6,Editor,2012.0
7,Nice Answer,2012.0
8,Yearling,2012.0
9,Nice Question,2012.0


In [17]:
# From "Badges", for "UserId" equal to 32,
# count how many unique "Name" types there are (group by "Name").
# Also, for each "Name" type, compute the min, average, and max per year.
# Return only the top four "Count" values.
pd.read_sql_query(
    """
    SELECT 
        Name,
        COUNT(*) AS Count,
        MIN(CAST(strftime('%Y', Date) AS FLOAT)) AS MinYear,
        AVG(CAST(strftime('%Y', Date) AS FLOAT)) AS MeanYear,
        MAX(CAST(strftime('%Y', Date) AS FLOAT)) AS MaxYear
    FROM Badges
    WHERE UserId=23
    Group BY Name
    ORDER BY Count DESC
    LIMIT 4
    """,
    conn
)

,Name,Count,MinYear,MeanYear,MaxYear
0,Nice Question,4,2011.0,2013.25,2017.0
1,Yearling,3,2012.0,2013.00,2014.0
2,Popular Question,3,2014.0,2014.00,2014.0
3,Notable Question,2,2015.0,2015.00,2015.0


In [18]:
# Join "Tags", "Posts", and "Users",
# for all rows where "OwnerUserId" is not equal to -1
# Return the top six records with respect to "Count" column in "Tags"
# and column "TagName" in "Tags".
pd.read_sql_query(
    """
    SELECT
        Tags.TagName, Tags.Count,
        Posts.OwnerUserId,
        Users.Age, Users.Location, Users.DisplayName
    FROM Tags
    JOIN Posts ON Posts.Id=Tags.WikiPostId
    JOIN Users ON Users.AccountId=Posts.OwnerUserId
    WHERE OwnerUserId != -1
    ORDER BY Tags.Count DESC, Tags.TagName ASC
    LIMIT 6
    """,
    conn
)

,TagName,Count,OwnerUserId,Age,Location,DisplayName
0,canada,802,101.0,34.0,"Mumbai, India",hitec
1,europe,681,583.0,35.0,"Philadelphia, PA",Adam Tuttle
2,visa-refusals,554,1737.0,34.0,"New York, NY",Benjamin Pollack
3,australia,411,101.0,34.0,"Mumbai, India",hitec
4,eu,204,583.0,35.0,"Philadelphia, PA",Adam Tuttle
5,new-york-city,204,101.0,34.0,"Mumbai, India",hitec


# Close the call to the data base

In [19]:
conn.close()

# Import JSON file


In [20]:
with open('data/parking_bay_sensors.json', 'r') as jsonfile:
     data_json = json.load(jsonfile)
     data_json_import = pd.json_normalize(data_json)
     data_json_import.drop(["location.lon", "location.lat"], axis=1, inplace=True)

In [21]:
data_json_import.loc[:5, :]

,lon,lat,st_marker_id,status,bay_id,parking_zone,last_updated
0,144.949484,-37.804403,9578,Present,None,7725,2023-03-28T03:37:57.550000+00:00
1,144.949557,-37.803989,9584,Unoccupied,None,7725,2023-03-28T03:37:57.550000+00:00
2,144.949566,-37.803936,9585,Present,None,7725,2023-03-28T03:37:57.550000+00:00
3,144.949575,-37.803884,9586,Present,None,7725,2023-03-28T03:37:57.551000+00:00
4,144.949584,-37.803833,9587,Present,None,7725,2023-03-28T03:37:57.551000+00:00
5,144.949594,-37.803779,9588,Unoccupied,None,7725,2023-03-28T03:37:57.551000+00:00


In [22]:
data_json[:2]

[{'location': {'lon': 144.9494842784271, 'lat': -37.80440338694197},
  'lon': 144.9494842784271,
  'lat': -37.80440338694197,
  'st_marker_id': 9578,
  'status': 'Present',
  'bay_id': None,
  'parking_zone': 7725,
  'last_updated': '2023-03-28T03:37:57.550000+00:00'},
 {'location': {'lon': 144.94955668232427, 'lat': -37.80398867559876},
  'lon': 144.94955668232427,
  'lat': -37.80398867559876,
  'st_marker_id': 9584,
  'status': 'Unoccupied',
  'bay_id': None,
  'parking_zone': 7725,
  'last_updated': '2023-03-28T03:37:57.550000+00:00'}]

# Import XML file

In [23]:
def parse_XML(xml_file, df_cols): 
    """Load a XML file and convert it to pandas DataFrame.
    The first element of df_cols is supposed to be the identifier variable. 
    """
    
    xtree = et.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        res.append(node.attrib.get(df_cols[0])) # add ID variable
        for el in df_cols[1:]: 
            if (node is not None) and (node.find(el) is not None):
                # Add other variables
                res.append(node.find(el).text)
            else: 
                # Add "None" for NA values
                res.append(None)
        rows.append({df_cols[i]: res[i] for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [24]:
data_xml = parse_XML(
    "data/uvvalues.xml",
    ["id", "name", "index", "time", "date", "fulldate", "utcdatetime", "status"])

In [25]:
data_xml.loc[:6, :]

,id,name,index,time,date,fulldate,utcdatetime,status
0,Adelaide,adl,0.0,6:47 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok
1,Alice Springs,ali,0.0,6:47 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok
2,Brisbane,bri,0.0,7:17 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok
3,Canberra,can,0.0,7:17 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok
4,Casey,cas,0.0,5:17 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok
5,Darwin,dar,0.0,6:47 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok
6,Davis,dav,0.0,4:17 PM,14/06/2023,"Wednesday, 14 June 2023",2023/06/14 09:17,ok


# Import from HTML (web)

In [26]:
URL = "https://en.wikipedia.org/wiki/List_of_20th-century_classical_composers"
data_wiki = pd.read_html(URL)

In [27]:
len(data_wiki)

3

In [28]:
data_wiki[0]

,Lists of classical composers by era and century,Unnamed: 1
0,Medieval (500–1400) Renaissance (1400–1600) Ba...,NaN
1,Medieval,(500–1400)
2,Renaissance,(1400–1600)
3,Baroque,(1600–1760)
4,Classical,(1730–1820)
5,Romantic,(1815–1910)
6,Modernist,(1890–1950)
7,Postmodernist,(since 1930)
8,20th century,(1901–1999)
9,21st century,(since 2000)


In [29]:
data_wiki[1]

,0,1
0,Medieval,(500–1400)
1,Renaissance,(1400–1600)
2,Baroque,(1600–1760)
3,Classical,(1730–1820)
4,Romantic,(1815–1910)
5,Modernist,(1890–1950)
6,Postmodernist,(since 1930)
7,20th century,(1901–1999)
8,21st century,(since 2000)
9,NaN,vte


In [30]:
data_wiki[2].loc[:5, :]

,Name,Year of birth,Year of death,Nationality,Notable 20th-century works,Remarks
0,Charles Dancla,1817,1907.0,French,"Solo de concours no. 7, Op. 224",Romanticism
1,Luigi Arditi,1822,1903.0,Italian,NaN,NaN
2,Theodor Kirchner,1823,1903.0,German,NaN,NaN
3,Carl Reinecke,1824,1910.0,German,"Trio for piano, clarinet and horn in B♭, Op. 2...",Romanticism
4,Richard Hol,1825,1904.0,Dutch,Organ music,Romanticism
5,Ludwig Minkus,1826,1917.0,Austrian,NaN,NaN


# Many files

In [31]:
files = os.listdir()
files

['.vscode',
 'Ch_10.ipynb',
 'Ch_11.ipynb',
 'Ch_12.ipynb',
 'Ch_13.ipynb',
 'Ch_4.ipynb',
 'Ch_5.ipynb',
 'Ch_6.ipynb',
 'Ch_7.ipynb',
 'Ch_8.ipynb',
 'Ch_9.ipynb',
 'data',
 'template_packages.ipynb']

In [32]:
def print_file_size(wd):
    tot_size = 0
    for (root, dirs, files) in os.walk(wd):
        #print(f"PATH: {root} ")
        #print(f"DIRS: {dirs} ")
        # print(dirs)
        # print(files)
        folder_name = os.path.relpath(root)
        print(f"FOLDER: {folder_name}")
        for file in files:
            file_path = os.path.join(root, file)
            print(f"    {file:-<50} bytes size: {os.path.getsize(file_path)}")
            tot_size += os.path.getsize(file_path)
    
    print(f"\nTOTAL SIZE = {tot_size}")

In [33]:
print_file_size(os.getcwd())

FOLDER: .
    Ch_10.ipynb--------------------------------------- bytes size: 242736
    Ch_11.ipynb--------------------------------------- bytes size: 172623
    Ch_12.ipynb--------------------------------------- bytes size: 789883
    Ch_13.ipynb--------------------------------------- bytes size: 15629
    Ch_4.ipynb---------------------------------------- bytes size: 171216
    Ch_5.ipynb---------------------------------------- bytes size: 45249
    Ch_6.ipynb---------------------------------------- bytes size: 203585
    Ch_7.ipynb---------------------------------------- bytes size: 512289
    Ch_8.ipynb---------------------------------------- bytes size: 24211
    Ch_9.ipynb---------------------------------------- bytes size: 859763
    template_packages.ipynb--------------------------- bytes size: 2522
FOLDER: .vscode
    settings.json------------------------------------- bytes size: 53
FOLDER: data
    37_pzu_warsaw_marathon_simplified.csv------------- bytes size: 103256
    air_